In [322]:
import pandas as pd
from ggplot import *
import matplotlib.pyplot as plt
%matplotlib inline
import psycopg2
import scipy
import math
import pickle


#Scaling
normilize = True
scale = False

#Spliting features and classifiers
SUBSET_FEATURES = True
SUBMODELS = False

#Showing Each classifier
SEPERATE_CLASSIFIERS = True

#Combining output
NN_COMBINE = True
VOTING_CLASSIFIERS = True


pd.set_option('display.max_columns', None)

try:
    conn = psycopg2.connect("dbname='capstone' user='samkreter' host='localhost'")
except:
    print("I am unable to connect to the database")

cur = conn.cursor()



sql = "select   fights.f1result, \
                f1.height_inches AS f1_height,\
                f1.reach_inches AS f1_reach, \
                f1.weight_lbs AS f1_weight, \
                f1.strike_offense_per_min AS f1_strike_offense_per_min, \
                f1.strike_offense_accuracy AS f1_strike_offense_accuracy, \
                f1.strike_defense_per_min AS f1_strike_defense_per_min, \
                f1.strike_defense_accuracy AS f1_strike_defense_accuracy, \
                f1.takedowns_per_fight AS f1_takedowns_per_fight, \
                f1.takedowns_accuracy AS f1_takedowns_accuracy, \
                f1.takedowns_defense AS f1_takedowns_defense, \
                f1.submissions_per_fight AS f1_submissions_per_fight, \
                f1.total_fights AS f1_total_fights, \
                f1.association AS f1_association, \
                f1.country AS f1_country, \
                f1.wins AS f1_wins, \
                f1.losses AS f1_losses, \
                f1.total_fights AS f1_total_fights, \
                                                \
                f2.height_inches AS f2_height, \
                f2.reach_inches AS f2_reach, \
                f2.weight_lbs AS f2_weight, \
                f2.strike_offense_per_min AS f2_strike_offense_per_min, \
                f2.strike_offense_accuracy AS f2_strike_offense_accuracy, \
                f2.strike_defense_per_min AS f2_strike_defense_per_min, \
                f2.strike_defense_accuracy AS f2_strike_defense_accuracy, \
                f2.takedowns_per_fight AS f2_takedowns_per_fight, \
                f2.takedowns_accuracy AS f2_takedowns_accuracy, \
                f2.takedowns_defense AS f2_takedowns_defense, \
                f2.submissions_per_fight AS f2_submissions_per_fight, \
                f2.total_fights AS f2_total_fights, \
                f2.association AS f2_association, \
                f2.country AS f2_country, \
                f2.wins AS f2_wins, \
                f2.losses AS f2_losses, \
                f2.total_fights AS f2_total_fights \
                from octagon.fights INNER JOIN octagon.fighters AS f1 ON octagon.fights.f1name = f1.name INNER JOIN octagon.fighters AS f2 ON octagon.fights.f2name = f2.name;"

df = pd.read_sql_query(sql, conn)

test = pd.read_sql_query("select * from octagon.fighters",conn)



# df = pd.read_sql_query("select * from octagon.fights",conn)
df2 = pd.read_sql_query("select * from octagon.fighters",conn)
conn.close()


In [323]:

# print(test['reach_inches'].isnull().sum())
test = test.interpolate()
test = test.dropna()
test.head()
# print(test['reach_inches'].isnull().sum())

,fid,name,dob,height_inches,weight_lbs,association,division,country,reach_inches,strike_offense_per_min,strike_offense_accuracy,strike_defense_per_min,strike_defense_accuracy,takedowns_per_fight,takedowns_accuracy,takedowns_defense,submissions_per_fight,wins,losses,draws,total_fights,win_pct,finish_pct
5,12078,Mizuto Hirota,1981-05-05,67,145,Cave,Featherweight,Japan,67.000000,3.490,45.0,3.250,53.0,2.160,68.0,64,0.00,0,2,1,3,0,0
6,258,Alex Andrade,1974-05-14,71,200,Lion's Den,Light Heavyweight,United States,69.000000,0.200,36.0,2.600,53.0,0.000,0.0,25,0.80,0,0,0,0,0,0
7,87012,Cesar Arzamendia,1991-01-31,71,155,Paraguay Combat Team,Featherweight,Paraguay,71.000000,5.370,45.0,5.200,56.0,7.320,75.0,0,2.40,0,0,0,0,0,0
10,186,Mikey Burnett,1974-07-16,66,170,Lion's Den,Welterweight,USA,71.857143,0.824,17.6,0.696,13.6,0.976,16.8,0,0.64,0,0,0,0,0,0
13,174,Dan Bobish,1970-01-26,73,345,Strong Style Fight Team,Super Heavyweight,United States,72.714286,2.060,44.0,1.740,34.0,2.440,42.0,0,1.60,0,0,0,0,0,0


### Convert text ones into numeric with normilization

In [324]:


country_names = list((df['f1_country'].append(df['f2_country'])).unique())
df['f1_country'] = df['f1_country'].apply(lambda x: country_names.index(x) / len(country_names))
df['f2_country'] = df['f2_country'].apply(lambda x: country_names.index(x) / len(country_names))


asso_names = list((df['f1_association'].append(df['f2_association'])).unique())
df['f1_association'] = df['f1_association'].apply(lambda x: asso_names.index(x) / len(asso_names))
df['f2_association'] = df['f2_association'].apply(lambda x: asso_names.index(x) / len(asso_names))



# df['f1_association'] = df['f1_association'].apply(lambda x: names.index(x))
# df['f2_association'] = df['f2_association'].apply(lambda x: names.index(x))

df.head()

,f1result,f1_height,f1_reach,f1_weight,f1_strike_offense_per_min,f1_strike_offense_accuracy,f1_strike_defense_per_min,f1_strike_defense_accuracy,f1_takedowns_per_fight,f1_takedowns_accuracy,f1_takedowns_defense,f1_submissions_per_fight,f1_total_fights,f1_association,f1_country,f1_wins,f1_losses,f1_total_fights,f2_height,f2_reach,f2_weight,f2_strike_offense_per_min,f2_strike_offense_accuracy,f2_strike_defense_per_min,f2_strike_defense_accuracy,f2_takedowns_per_fight,f2_takedowns_accuracy,f2_takedowns_defense,f2_submissions_per_fight,f2_total_fights,f2_association,f2_country,f2_wins,f2_losses,f2_total_fights
0,win,78,81,260,2.82,34,2.46,46,3.26,66,50,2.2,4,0.000000,0.000000,1,3,4,79,81,265,0.85,44,2.48,52,2.12,40,33,4.2,3,0.343750,0,1,2,3
1,win,70,NaN,151,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7,0.001645,0.000000,4,3,7,70,73,170,1.79,36,2.21,63,2.51,48,64,0.4,25,0.093750,0,15,10,25
2,win,72,NaN,170,1.89,36,3.23,52,0.62,16,58,1.5,3,0.003289,0.000000,1,2,3,68,66,155,2.39,38,2.37,57,2.64,42,86,1.1,3,0.182566,0,1,2,3
3,win,69,74,247,0.97,30,2.87,59,1.06,12,66,0.7,7,0.004934,0.000000,4,3,7,74,NaN,235,2.61,69,3.91,26,4.34,57,66,4.3,2,0.309211,0,0,2,2
4,win,72,73,170,1.61,41,2.00,46,4.17,70,45,0.6,9,0.006579,0.016667,4,5,9,74,NaN,170,2.55,40,2.34,35,0.00,0,0,0.0,1,0.003289,0,0,1,1


In [ ]:
df['f1result'] = df['f1result'].replace(['win', 'draw', 'NC'],[1,2,2])
df.head()

### Subsetting the features to improve performance

In [ ]:
if SUBSET_FEATURES:
    subset = ['f1result','f1_strike_offense_per_min','f1_strike_defense_per_min','f1_association','f1_wins','f1_losses', 
        'f2_strike_offense_per_min','f2_strike_defense_per_min','f2_association','f2_wins','f2_losses']
    df = df[subset]

### Interpolate the data to fill in the NAs to get a better result of the data
It was found that dropna works the best for the highest correclation results, We will revisit this later

In [ ]:
df0 = df.interpolate()
df0 = df.interpolate(method='spline', order=2)
df = df.interpolate(method='pchip')
df = df.dropna()

df.head()

## Move the lossers to the end to have data for losing people

In [ ]:
df.corr()

In [ ]:
f1_end = math.ceil(len(df.columns) / 2)
df2 = df.copy()
tmp = df2.ix[:,1:f1_end]


df2.ix[:,1:f1_end] = df2.ix[:,f1_end:].values
df2.ix[:,f1_end:] = tmp.values
df2['f1result'] = 0


df2.head()

### Combine the two dataframes and randomize

In [ ]:
df = df.append(df2)
df = df.sample(frac=1).reset_index(drop=True)
df.head()

In [ ]:
#Pull the labels out of the dataset
y = df['f1result']
del df['f1result']

X = df

In [ ]:
from sklearn import datasets
# Our Classifier
from sklearn.naive_bayes import GaussianNB
# Get the accuracy score of the final model
from sklearn.metrics import accuracy_score
# Split the training set into train and test
from sklearn.cross_validation import train_test_split
#MLP Classifer
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, VotingClassifier
from sklearn import preprocessing
from sklearn.feature_selection import RFE

if normilize:
    X = preprocessing.normalize(X, norm='l2')
    
if scale:
    X = preprocessing.scale(X)

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=.2,random_state = 1)

names = [
         "MLP",
         "Naive Bayes",
         "KNN",
         "SVM Linear",
         "SVM gamma",
         "Decsion Tree", 
         "Random Forest",
         "adaBoost"
        ]

clfs = [
        MLPClassifier(solver='sgd', alpha=1e-5, hidden_layer_sizes=(5, 2), random_state=1,max_iter=400),
        GaussianNB(),
        KNeighborsClassifier(3),
        SVC(kernel="linear", C=0.025,probability=True),
        SVC(gamma=2, C=1,probability=True),
        DecisionTreeClassifier(max_depth=5),
        RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
        AdaBoostClassifier()
       ]

if SUBMODELS and SUBSET_FEATURES:
    #feats = [0,2,3]
    feats = [0,2,3,4,5,6,7]
    clfs = [clfs[i] for i in feats]
    names = [names[i] for i in feats]
elif SUBMODELS:
    feat = [3,5,7]
    clfs = [clfs[i] for i in feats]
    names = [names[i] for i in feats]

#predict_proba to get the probabilities 
    
    #All Features
# SVC- Linear, DTree and AdaBoost are the best especially with Voter
 
#Subset
# MLP, KNN, SVM Linear 
# Normilization Good 
    
    
#13, 15, 16, 30, 
# print("NBC Accuracy of the model is: %.2f%%" % (accuracy_score(nbc_preds,y_test) * 100))
# print("MLP Accuracy of the model is: %.2f%%" % (accuracy_score(mlp_preds,y_test) * 100))


## Finding the importance of each feature using Recursion

In [ ]:
if False:
    f1_end = int(len(df.columns) / 2)
    indexs = np.array([i for i in range(f1_end)])
    selector = RFE(clfs[3], 14, step=1)
    selector = selector.fit(X_train, y_train)
    print(selector.support_)
    print(indexs[selector.support_[:f1_end]])
    print(indexs[selector.support_[f1_end:]])
#

## Looking at each Classifier separatly 

In [ ]:
if SEPERATE_CLASSIFIERS:
    
    pred_list_train = []
    pred_list_test = []
    
    for index, clf in enumerate(clfs):
        clf.fit(X_train,y_train)

        preds = clf.predict(X_test)
        
        preds_train = np.amax(clf.predict_proba(X_train),axis=1)
        preds_test = np.amax(clf.predict_proba(X_test),axis=1)
        
        pred_list_train.append(preds_train)
        pred_list_test.append(preds_test)
        #print(names[index] + " Accuracy of the model is: %.2f%%" % (accuracy_score(preds,y_test) * 100))
        print("%.2f" % (accuracy_score(preds,y_test) * 100))

#     with open("clfs-2.pickle","wb") as f:
#         pickle.dump(clfs,f)

## Using Voting for Combination

In [ ]:
if VOTING_CLASSIFIERS:
    clf_vote = VotingClassifier(estimators=list(zip(names,clfs)),voting='soft')
    clf_vote.fit(X_train,y_train)
    preds = clf_vote.predict(X_test)
    print("%.2f" % (accuracy_score(preds,y_test) * 100))


### Testing Combination of Classifiers using NN - DOES NOT WORK GOOD

In [ ]:
if NN_COMBINE:
    combine_clf = MLPClassifier(solver='sgd', alpha=1e-5, hidden_layer_sizes=(5, 2), random_state=1)
    combine_clf.fit(np.array(pred_list_train).T,y_train)
    preds = combine_clf.predict(np.array(pred_list_test).T)
    print("%.2f" % (accuracy_score(preds,y_test) * 100))



In [ ]:

try:
    conn = psycopg2.connect("dbname='capstone' user='samkreter' host='localhost'")
except:
    print("I am unable to connect to the database")

cur = conn.cursor()

sql = "select fid, \
            height_inches, \
            reach_inches, \
            weight_lbs, \
            strike_offense_per_min, \
            strike_offense_accuracy, \
            strike_defense_per_min, \
            strike_defense_accuracy, \
            takedowns_per_fight, \
            takedowns_accuracy, \
            takedowns_defense, \
            submissions_per_fight, \
            total_fights, \
            association, \
            country, \
            wins, \
            losses, \
            total_fights FROM octagon.fighters"

#16635
#230

#df1 = pd.read_sql_query(sql + str(16635), conn)
df = pd.read_sql_query("select * from octagon.fighters", conn)

conn.close()


In [ ]:
#Figure out good interpolator without having to load the entire database each time
df = df.interpolate()

f1 = df[df['fid'] == 230].drop('fid',axis=1)
f2 = df[df['fid'] == 230].drop('fid',axis=1)

if not f1.empty and not f2.empty:
    print("hello")
#np.append(f1,f2)

